# Part 2: Creating your first Experiment in CRIPT 
---

In this notebook we will create document our first experiment, and all the associated data. The following example is the anionic polymerization of styrene with secBuLi in a mixture of THF and toluene. The reaction is then quenched with butanol and precipitated into methanol to obtain the final product, polystyrene.

![Polystyrene_synthesis](../Jupyter_notebook/supporting_files/polystyrene_synthesis.svg)


---

Before diving directly into this examples we will start with the basics: importing cript module, and connecting to the CRIPT database.

<u>Action:</u> Replace 'user = ###@###.##' with the email address you used for your `User node` or user id. Then run the cell. <span style="color:gray">(You can run a cell by either clicking on it and hitting the 'play/run' button in the toolbar or click the cell and hit "shift" + "enter".)</span>

<u>Output:</u> You should see "Import sucessful!", "Connection to database 'test' successful.", "you logged in sucessfully." 

In [4]:
import cript
print("Import successful!")

# Connect to database
db_username = "DW_cript"
db_password = "YXMaoE1"
db_project = "cript_testing"
db_database = "test"
user = "johndoe1c3@cript.edu"  # Put your email or uid here
db = cript.CriptDB(db_username, db_password, db_project, db_database, user)

Import successful!
Connection to database 'test' successful.


CRIPTError: johndoe1c3@cript.edu not found in database.


<span style="color:SteelBlue"> This previous block of code is the standard code you should type first for all future projects. </span> 

---

In [ ]:
collections = db.view_all("collection")

In [ ]:
db.collections


---
---

## Key Takeaways

* You have 

---
---

### What's next?
